In [1]:
import geopandas as gpd
from folium import (
    LayerControl,
    Element,
    TileLayer,
)
import plotly.express as px
from os import path


from core.geo import areal_weighted_interpolation
from core.urbanismo import prepare_tracts, prepare_risk_area

# Carregando os dados extraídos no notebook anterior

Neste notebook, vamos utilizar os dados extraídos e salvos pelo notebook [urbanismo/urbanismo - extração.ipynb](https://github.com/ObservatorioPoliticasTCM/dados-painel/blob/main/notebooks/urbanismo/urbanismo%20-%20extra%C3%A7%C3%A3o.ipynb).

In [2]:
input_dir = output_dir = path.join('dados', 'urbanismo')

In [ ]:
df_hid = gpd.read_file(path.join(input_dir, 'risco_hidrologico_original.gpkg'))
df_hid


In [ ]:
df_geo = gpd.read_file(path.join(input_dir, 'risco_geologico_original.gpkg'))
df_geo


In [ ]:
df_censo = gpd.read_file(path.join(input_dir, 'setores_censitarios_original.gpkg'))
df_censo

In [ ]:
df_censo.crs

In [ ]:
df_veg = gpd.read_file(path.join(input_dir, 'cobertura_vegetal_original.gpkg'))
df_veg

In [ ]:
df_quadras = gpd.read_file(path.join(input_dir, 'quadras_viarias_original.gpkg'))
df_quadras

In [ ]:
df_subs = gpd.read_file(path.join(input_dir, 'subprefeituras_original.gpkg'))
df_subs

# Calculando a população de cada área de risco

Primeiro, vamos inspecionar visualmente os geodataframes.

In [ ]:
df_geo.explore()

In [ ]:
df_censo.iloc[:500].explore()

In [ ]:
df_veg.iloc[:500].explore()

## Calculando as interseções

1. Ajustar os polígonos dos setores censitários:
    1. Remover as áreas de vegetação dos setores censitarios;
    1. Calcular a interseção com as quadras viárias (para remover as ruas);
    1. Calcular a área de cada setor censitário ajustado (nova área total);
1. Ajustar os polígonos das áreas de risco;
    1. Calcular a interseção das áreas de risco com as subprefeituras;
1. Realizar a interpolação dos polígonos:
    1. Calcular a interseção das áreas de risco ajustadas e setores censitários ajustados;
    1. Calcular o peso da interseção em relação ao setor ajustado;
    1. Calcular a população e moradias com base no peso;
    1. Agregar população e moradia por área de risco;
    1. Agregar população e moradia por subprefeitura e grau de risco;

In [ ]:
df_censo['NM_DIST'].value_counts()

### Testando com o distrito de Grajaú

#### Filtrando os geodataframes

In [ ]:
df_censo_filtrado = df_censo[df_censo['NM_DIST']=='Grajaú']
df_censo_filtrado

Primeiro, vamos filtrar a área de vegetação para os tipos mais adequados para o ajuste dos setores censitários.

In [ ]:
df_veg_filtrada = df_veg[df_veg.intersects(df_censo_filtrado.union_all())]

df_veg_filtrada

#### Ajustando os polígonos dos setores censitários

##### Removendo a vegetação

Depois, calculamos a diferença entre os setores censitários e a vegetação, para remover as área não povoadas.

In [ ]:
ol1 = gpd.overlay(df_censo_filtrado, df_veg_filtrada,
                  how='difference',
                  keep_geom_type=True)

ol1.head()

In [17]:
# tooltip_columns = ['CD_SETOR', 'v0001', 'v0002', 'v0003', 'v0004', 'v0005', 'v0006', 'v0007']

# m = df_censo_filtrado.explore(
#     tiles='CartoDB positron',
#     color='blue',  # Cor dos setores censitários
#     style_kwds={'fillOpacity': 0.2},  # Transparência do preenchimento
#     name='Setores Censitários Originais',
#     tooltip=tooltip_columns
# )
# m = ol1.explore(
#     m=m,
#     color='purple',  # Cor das interseções
#     style_kwds={'fillOpacity': 0.5},  # Transparência do preenchimento
#     name='Setores Censitários Ajustados',
#     tooltip=tooltip_columns
# )

# # Adicionar camada de satélite
# TileLayer(
#     tiles='https://mt1.google.com/vt/lyrs=s&x={x}&y={y}&z={z}',
#     attr='Google Satellite',
#     name='Google Satellite'
# ).add_to(m)

# # Adicionar controle de camadas
# LayerControl().add_to(m)

# # Criar a legenda personalizada
# legend_html = """
# <div style="position: fixed; 
#             bottom: 50px; left: 50px; width: 200px; height: 120px; 
#             background-color: white; z-index:1000; padding: 10px; 
#             border: 2px solid grey; border-radius: 5px;">
#     <h4>Legenda</h4>
#     <i style="background: blue; width: 10px; height: 10px; display: inline-block;"></i> Setores Censitários Originais<br>
#     <i style="background: purple; width: 10px; height: 10px; display: inline-block;"></i> Setores Censitários Ajustados<br>
# </div>
# """
# m.get_root().html.add_child(Element(legend_html))

# m

##### Removendo a malha viária

Depois, removemos a malha viária dos polígonos calculando a interseção com as quadras viárias.

In [ ]:
ol2 = gpd.overlay(ol1, df_quadras[['geometry']],
            how='intersection',
            keep_geom_type=True)
ol2

In [ ]:
ol2.iloc[:500].explore()

Como nosso interesse é pelo setor censitário, vamos agrupar as interseções por setor censitário. Além disso, precisaremos da área do setor ajustado para os cálculos de ponderação, vamos adicionar a coluna agora.

In [ ]:
ol2 = (
    ol2
    .dissolve(by='CD_SETOR', aggfunc='first')
    .reset_index()
)

ol2['area_setor_ajustado'] = ol2.area
ol2

#### Ajustando os polígonos das áreas de risco

##### Calculando a interseção das áreas de risco e subprefeituras

In [ ]:
ol3 = df_geo.overlay(
    df_subs[['cd_identificador_subprefeitura',
                 'cd_subprefeitura',
                 'nm_subprefeitura',
                 'geometry']],
    how='intersection',
    keep_geom_type=True
)

ol3

In [ ]:
ol3.loc[:, 'id_area_subprefeitura'] = ol3.loc[:, 'id'].astype(str) + '.subpref.' + ol3.loc[:, 'cd_identificador_subprefeitura'].astype(str)
ol3

#### Realizando a interpolação

##### Calculando a interseção com as áreas de risco

Depois, precisaremos calcular a interseção de cada um dos dataframes de risco com os setores censitários ajustados.

In [ ]:
ol4 = gpd.overlay(
    ol2.loc[:, ['CD_SETOR', 'v0001', 'v0002', 'v0003', 'v0004',
                'v0005', 'v0006', 'v0007', 'area_setor_ajustado', 'geometry']],
    ol3,
    how='intersection',
    keep_geom_type=True)

ol4.head()

Vamos avaliar visualmente o resultado da interseção com base na primeira área de risco.

In [ ]:
id_area = ol4.loc[:, 'id'].iloc[0]
id_area

In [ ]:
cd_setor_list = ol4.loc[ol4['id']==id_area, 'CD_SETOR'].tolist()
ol4.loc[ol4['id']==id_area, ['id', 'CD_SETOR']]

In [ ]:
m = df_censo[df_censo['CD_SETOR'].isin(
    cd_setor_list)].explore(name='setor censitário', color='navy')

m = ol2[ol2['CD_SETOR'].isin(cd_setor_list)].explore(
    m = m, name='setor censitário ajustado')

m = df_geo[df_geo['id'] == id_area].explore(
    m=m, color='orange', name='area de risco')

filtered_ol3 = ol4.loc[ol4['id'] == id_area, [
    'id', 'CD_SETOR', 'geometry']]
m = filtered_ol3.explore(m=m, color='purple', name='interseção')

LayerControl().add_to(m)

m

Na inspeção visual, nota-se que algumas interseções não aparentam representar áreas com moradias, mas simplesmente leves discrepâncias no desenho dos polígonos sobre áreas não populadas (ruas,  canteiros, etc.).

Uma opção de passo adicional seria explodir todos os polígonos e aplicar um buffer negativo para que esses recortes sem significado prático fossem removidos. Porém, como as áreas relativas a essas áreas também devem causar pouca distorção nos valores, não realizaremos essa remoção neste momento.

Antes de passar para os cálculos, vamos adicionar uma coluna com a área de cada interseção ao geodataframe.

In [ ]:
ol4['area_intersecao'] = ol4['geometry'].area
ol4

##### Calculando a proporção das áreas

O primeiro passo é calcular a proporção entre a área da interseção e a área do setor censitário.

In [ ]:
ol4['prop_setor'] = ol4['area_intersecao']/ol4['area_setor_ajustado']
ol4.head()

Sabendo que a proporção da interseção não deve ser maior do que 1, vamos conferir os valores de interseção.

In [ ]:
ol4['prop_setor'].hist()

Como a distribuição apresenta muitos valores abaixo de 0.1, vamos analisar esses casos visualmente para confirmar que fazem sentido.

In [30]:
filtro_intersecoes_pequenas = ol4['prop_setor']<=0.1
setores_02 = ol4.loc[filtro_intersecoes_pequenas, 'CD_SETOR'].tolist()
areas_risco_02 = ol4.loc[filtro_intersecoes_pequenas, 'id'].tolist()

In [ ]:
# Criar o mapa com as camadas
m = df_censo[df_censo['CD_SETOR'].isin(setores_02)].explore(name='Setor Censitário')

m = df_geo[df_geo['id'].isin(areas_risco_02)].explore(
    m=m, color='orange', name='Área de Risco')

filtered_ol3 = ol4.loc[filtro_intersecoes_pequenas, ['id', 'CD_SETOR', 'geometry', 'prop_setor']]
m = filtered_ol3.explore(m=m, color='purple', name='Interseção')

LayerControl().add_to(m)

# Adicionar a legenda personalizada
legend_html = """
<div style="position: fixed; 
            bottom: 50px; left: 50px; width: 200px; height: 120px; 
            background-color: white; z-index:1000; padding: 10px; 
            border: 2px solid grey; border-radius: 5px;">
    <h4>Legenda</h4>
    <i style="background: orange; width: 10px; height: 10px; display: inline-block;"></i> Área de Risco<br>
    <i style="background: purple; width: 10px; height: 10px; display: inline-block;"></i> Interseção<br>
    <i style="background: blue; width: 10px; height: 10px; display: inline-block;"></i> Setor Censitário
</div>
"""
m.get_root().html.add_child(Element(legend_html))

m

A inspeção visual mostra que parte das interseções não aparentam ser significativas, mas não causarão distorção relevante no cálculo, devido ao fator de pontederação baixo. Outras, entretanto, aparentam ser significativas, mas se tratam de interseções pequenas em comparação com os setores censitários, ocorrendo principalmente em setores sensitários maiores.

##### Calculando a população

O primeiro passo é calcular a população correspondente a cada interseção.

In [ ]:
ol4['pop_estimada'] = ol4['v0001']*ol4['prop_setor']

(
    ol4
    .loc[:, ['id','CD_SETOR','v0001', 'prop_setor', 'pop_estimada']]
    .sort_values('pop_estimada', ascending=False)
)

Depois, calculamos a população agregada por área de risco.

In [ ]:
area_risco_pop = (
    ol4
    .loc[:, ['id', 'pop_estimada']]
    .groupby('id')
    .sum()
    .round(0)
    .reset_index()
)

area_risco_pop['pop_estimada'] = area_risco_pop['pop_estimada'].astype(int)
area_risco_pop

Finalmente, trazemos a coluna de população para o dataframe original.

In [ ]:
df_geo_pop = df_geo.merge(
    area_risco_pop,
    how='inner'
)

df_geo_pop.head()

##### Visualizando a população

Para visualizar o resultado, vamos criar algumas visualizações. Primeiro, vamos criar dois *treemaps* por tipo de processo e grau de risco, alterando a ordem da hierarquia.

In [ ]:
df_tree = (
    df_geo_pop
    .groupby(['tx_tipo_processo_geologico', 'tx_grau_de_risco_geologico'], as_index=False)
    ['pop_estimada']
    .sum()
)

df_tree

In [ ]:
# Criar o gráfico do tipo treeview
fig = px.treemap(
    df_tree,
    path=['tx_grau_de_risco_geologico', 'tx_tipo_processo_geologico'],  # Caminho hierárquico
    values='pop_estimada',               # Valores para o tamanho das caixas
    title='Somatório de População Estimada por Tipo de Processo Geológico'
)

# Exibir o gráfico
fig.show()

In [ ]:
# Criar o gráfico do tipo treeview
fig = px.treemap(
    df_tree,
    path=['tx_tipo_processo_geologico', 'tx_grau_de_risco_geologico'],  # Caminho hierárquico
    values='pop_estimada',               # Valores para o tamanho das caixas
    title='Somatório de População Estimada por Tipo de Processo Geológico'
)

# Exibir o gráfico
fig.show()

Por último, vamos criar um mapa coroplético das áreas de risco, adicionando as camadas de setores censitários envolvidos, interseções envolvidas (e população atribuida) e adicionar também uma camada de satelite.

In [ ]:

# Filtrar os setores censitários presentes em ol3
setores_presentes = df_censo[df_censo['CD_SETOR'].isin(ol4['CD_SETOR'].unique())]
# Adicionar os setores censitários ao mapa com apenas o contorno
m = setores_presentes.explore(
    color=None,  # Remove o preenchimento
    style_kwds={'fill': False, 'color': 'skyblue'},  # Define apenas o contorno azul
    name='Setores Censitários',
    tiles='CartoDB positron'  # Mapa base
)

# Criar o mapa base
m = df_geo_pop.explore(
    m=m,
    column='pop_estimada',  # Coluna para o mapa coroplético
    cmap='YlOrRd',          # Paleta de cores
    legend=True,            # Adicionar legenda
    legend_kwds={
        'caption': 'População total',
        'position': 'bottomright',  # Alterar posição da legenda
        'frameon': True             # Adicionar quadro de fundo
    },
    tooltip=['tx_grau_de_risco_geologico', 'tx_tipo_processo_geologico', 'pop_estimada'],  # Informações no tooltip
    name='Áreas de Risco'   # Nome da camada
)

# Adicionar as interseções ao mapa
m = ol4.explore(
    m=m,
    column='pop_estimada',  # Coluna para o mapa coroplético
    cmap='YlOrRd',          # Paleta de cores
    legend=True,            # Adicionar legenda
    legend_kwds={
        'caption': 'População parcial',
        'position': 'bottomright',  # Alterar posição da legenda
        'frameon': True             # Adicionar quadro de fundo
    },
    tooltip=['CD_SETOR', 'pop_estimada'],  # Informações no tooltip
    name='Interseções'      # Nome da camada
)

# Adicionar outra camada base (Google Satellite)
TileLayer(
    tiles='https://mt1.google.com/vt/lyrs=s&x={x}&y={y}&z={z}',
    attr='Google Satellite',
    name='Google Satellite'
).add_to(m)

# Adicionar controle de camadas
LayerControl(position='bottomleft').add_to(m)

# Exibir o mapa
m


In [39]:
m.save('plots/população em áreas de risco.html')

# Generalizando o cálculo para toda a cidade

Com base nos passos e na avaliação exposta anterioremente, foram criadas algumas funções no módulo `core` para generalizar a metodologia. São as funções `areal_weighted_interpolation`, de `core.geo`, e `prepare_tracts` e `prepare_risk_area`, de `core.urbanismo`.

## Preparando os setores censitários

In [ ]:
df_censo_ajustado = prepare_tracts(df_tracts=df_censo,
                                   df_vegetation=df_veg,
                                   df_street_blocks=df_quadras,
                                   tracts_id_col='CD_SETOR')
df_censo_ajustado.head()

## Preparando as áreas de risco

In [ ]:
df_hid_ajustado = prepare_risk_area(risk_area_gdf=df_hid,
                                    risk_area_id_col='id',
                                    risk_grade_col_prefix='tx_grau',
                                    active_risk_prefix='r',
                                    subprefeitura_gdf=df_subs,
                                    subprefeitura_id_col='cd_identificador_subprefeitura',
                                    subprefeitura_additional_cols=['cd_subprefeitura',
                                                                   'nm_subprefeitura'])
df_hid_ajustado

In [ ]:
df_geo_ajustado = prepare_risk_area(risk_area_gdf=df_geo,
                                    risk_area_id_col='id',
                                    risk_grade_col_prefix='tx_grau',
                                    active_risk_prefix='r',
                                    subprefeitura_gdf=df_subs,
                                    subprefeitura_id_col='cd_identificador_subprefeitura',
                                    subprefeitura_additional_cols=['cd_subprefeitura',
                                                                   'nm_subprefeitura'])
df_geo_ajustado

## Calculando a população para risco hidrológico

Com os setores censitários ajustados, vamos calcular a população para as áreas de risco hidrológico.

In [ ]:
df_hid_ajustado.columns

In [ ]:
df_hid_pop = areal_weighted_interpolation(
    df_censo_ajustado,
    df_hid_ajustado,
    'id_area_subprefeitura',
    'v0001',
    'populacao_estimada'
)

df_hid_pop.head()

In [ ]:
df_hid_sub = areal_weighted_interpolation(
    df_censo_ajustado,
    df_hid_pop,
    'id_area_subprefeitura',
    'v0002',
    'total_domicilios_estimado'
)

df_hid_sub.head()

## Calculando a população para risco geológico

Com os setores censitários ajustados, vamos calcular a população para as áreas de risco geológico.

In [ ]:
df_geo_ajustado.columns

In [ ]:
df_geo_pop = areal_weighted_interpolation(
    df_censo_ajustado,
    df_geo_ajustado,
    'id_area_subprefeitura',
    'v0001',
    'populacao_estimada'
)

df_geo_pop.head()

In [ ]:
df_geo_sub = areal_weighted_interpolation(
    df_censo_ajustado,
    df_geo_pop,
    'id_area_subprefeitura',
    'v0002',
    'total_domicilios_estimado'
)

df_geo_sub.head()

# Agrupando os valores por área de risco e subprefeitura

## Agrupando os valores por área de risco

In [ ]:
id_col = 'id'
var_cols = ['populacao_estimada','total_domicilios_estimado']

df_geo_agg = (
    df_geo_sub
    .loc[:, [id_col] + var_cols]
    .groupby(id_col)
    .sum()
    .reset_index()
    .merge(df_geo)
)

df_geo_final = df_geo.merge(df_geo_agg, how='left')
df_geo_final = df_geo_final[df_geo_final['id'].isin(df_geo_ajustado['id'].unique())]
df_geo_final

In [ ]:
id_col = 'id'
var_cols = ['populacao_estimada','total_domicilios_estimado']

df_hid_agg = (
    df_hid_sub
    .loc[:, [id_col] + var_cols]
    .groupby(id_col)
    .sum()
    .reset_index()
)

df_hid_final = df_hid.merge(df_hid_agg)
df_hid_final

## Agrupando os valores por subprefeitura

In [51]:
id_col = 'cd_identificador_subprefeitura'
var_cols = ['populacao_estimada','total_domicilios_estimado']
geo_var_cols = {'populacao_estimada': 'populacao_em_risco_geologico',
                'total_domicilios_estimado': 'domicilios_em_risco_geologico'}
hid_var_cols = {'populacao_estimada': 'populacao_em_risco_hidrologico',
                'total_domicilios_estimado': 'domicilios_em_risco_hidrologico'}

In [ ]:
df_hid_sub_agg = (
    df_hid_sub
    .loc[:, [id_col] + var_cols]
    .groupby(id_col)
    .sum()
    .reset_index()
    .rename(columns=hid_var_cols)
)

df_hid_sub_agg

In [ ]:
df_geo_sub_agg = (
    df_geo_sub
    .loc[:, [id_col] + var_cols]
    .groupby(id_col)
    .sum()
    .reset_index()
    .rename(columns=geo_var_cols)
)

df_geo_sub_agg

In [ ]:
df_subs_final = (
    df_subs
    .merge(df_hid_sub_agg, how='left')
    .merge(df_geo_sub_agg, how='left')
    .fillna(0)
)

df_subs_final

## Adicionando dados de subprefeitura às áreas de risco geológico

Para que os polígonos possam ser filtrados no Qlik Sense, eles precisam ser possuir uma coluna com o nome das subprefeituras com as quais eles possuem interseção. Para isso, vamos fazer a associação entre o dataframe final (com a população e domicílios estimados) e a camada de subprefeitura.

Obs: isso causará a repetição dos polígonos que intersectam mais de uma subprefeitura, portanto o novo dataframe não pode ser usado para agregações.

In [ ]:
df_geo_final = df_geo_final.sjoin(df_subs[['geometry', 'cd_subprefeitura', 'nm_subprefeitura']],
                   how='left',
                   predicate='intersects')

df_geo_final

Vamos conferir se todas as áreas de risco possuem pelo menos uma subprefeitura associada.

In [ ]:
df_geo_final[df_geo_final['nm_subprefeitura'].isna()]

Como nenhum registro possui valor nulo no nome da subprefeitura, sabemos que todos tem uma subprefeitura associada.

In [ ]:
df_geo_ajustado.shape

In [ ]:
df_geo_final.shape

In [ ]:
df_geo_final.shape[0] - df_geo_ajustado.shape[0]

Vemos também que todas as associações entre áreas de risco e subprefeituras também estavam presentes no dataframe utilizado para a interpolação, indicando que as associações encontradas espacialmente são as mesmas encontradas no cálculo das interseções.

# Exportando os arquivos

Finalmente, salvamos os arquivos com as camadas com os totais de população e domicílios.

In [60]:
output_dir = path.join('data_output', 'urbanismo')

for c, gdf in [('risco_hidrologico', df_hid_final),
               ('risco_geologico', df_geo_final)]:
    filename=path.join(output_dir, c)
    gdf.to_file(f'{filename}.gpkg', driver='GPKG', layer=c)

In [61]:
filename = 'populacao_risco_subprefeitura'
filepath=path.join(output_dir, filename)
df_subs_final.drop(columns='geometry').to_csv(f'{filepath}.csv', index=False)